In [1]:
# 目标：简单的神经网络
# 保存网络
# 读取数据，验证集测试模型
# 加入指数衰减
# 梯度下降
# L2正则
# 滑动平均模型
# dropout


In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os

In [9]:
mnist=input_data.read_data_sets('MNIST_data/',one_hot=True)
BATCH_SIZE=100
INPUT=784
OUTPUT=10
NUM_CHANNELS=1
IMAGE=28
LEARNING_RATE=0.001
TRAINING_STEPS=300

#初始化权重
def init_weight(shape):
    weights=tf.Variable(tf.truncated_normal(shape,stddev=0.1))
    return weights

#初始化偏执
def init_bias(shape):
    bias=tf.Variable(tf.constant(0.1,shape=shape))
    return bias

#卷积层封装
def con2d(x,weights,strides,padding_type):
    return tf.nn.conv2d(x,weights,strides=strides,padding=padding_type)

#池化层
def pooling(x,kernel,strides,padding_type):
    return tf.nn.max_pool(x,ksize=kernel,strides=strides,padding=padding_type)

x=tf.placeholder(tf.float32,shape=[None,INPUT],name='x')
y_=tf.placeholder(tf.float32,shape=[None,OUTPUT],name='y')
x_=tf.reshape(x,[-1,IMAGE,IMAGE,NUM_CHANNELS])

#卷积 # 第一层p=(f-1)/2=2 n+2p-f+1=28+2*2-5+1=28
weights_1=init_weight([5,5,1,32])
bias=init_bias([32])
layer1_conv=tf.nn.relu(con2d(x_,weights_1,[1,1,1,1],'SAME')+bias)

#池化 # 第二层p=(f-1)/2=1 n+2p-f+1=26+2*(1/2)-2+1=26  ==>28
layer2_pool=pooling(layer1_conv,[1,2,2,1],[1,2,2,1],'SAME')

# 再加一层卷积

weights_4=init_weight([3,3,1,64])
bias=init_bias([64])
layer4_conv=tf.nn.relu(con2d(x_,weights_4,[1,1,1,1],'SAME')+bias)
#全连接

node_nums=layer4_conv.get_shape().as_list()
weight_3=init_weight([node_nums[1]*node_nums[2]*node_nums[3],10])
bias_3=init_bias([10])
layer2_pool=tf.reshape(layer4_conv,[-1,node_nums[1]*node_nums[2]*node_nums[3]])
y=tf.nn.softmax(tf.matmul(layer2_pool,weight_3)+bias_3)

# 设置损失函数
cross_entropy=tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y),reduction_indices=[1]))
train_step=tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(cross_entropy)

correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
correct=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    xs,ys=mnist.train.next_batch(BATCH_SIZE)
    print(xs.shape,ys.shape)
    for i in range(TRAINING_STEPS):
        _,correct_,cross_entropy_=sess.run([train_step,correct,cross_entropy],feed_dict={x:xs,y_:ys})
    print(correct_,'-----------',cross_entropy_)
        
            

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
(100, 784) (100, 10)
0.75 ----------- 1.10251


In [1]:
# 保存模型
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os

mnist=input_data.read_data_sets('MNIST_data/',one_hot=True)
BATCH_SIZE=100
INPUT=784
OUTPUT=10
NUM_CHANNELS=1
IMAGE=28
LEARNING_RATE=0.001
TRAINING_STEPS=300
MODEL_PATH='CONV_Mnist'
MODEL_NAME='model.ckpt'
#初始化权重
def init_weight(shape):
    weights=tf.Variable(tf.truncated_normal(shape,stddev=0.1))
    return weights

#初始化偏执
def init_bias(shape):
    bias=tf.Variable(tf.constant(0.1,shape=shape))
    return bias

#卷积层封装
def con2d(x,weights,strides,padding_type):
    return tf.nn.conv2d(x,weights,strides=strides,padding=padding_type)

#池化层
def pooling(x,kernel,strides,padding_type):
    return tf.nn.max_pool(x,ksize=kernel,strides=strides,padding=padding_type)

x=tf.placeholder(tf.float32,shape=[None,INPUT],name='x')
y_=tf.placeholder(tf.float32,shape=[None,OUTPUT],name='y')
x_=tf.reshape(x,[-1,IMAGE,IMAGE,NUM_CHANNELS])

#卷积 # 第一层p=(f-1)/2=2 n+2p-f+1=28+2*2-5+1=28
weights_1=init_weight([5,5,1,32])
bias=init_bias([32])
layer1_conv=tf.nn.relu(con2d(x_,weights_1,[1,1,1,1],'SAME')+bias)

#池化 # 第二层p=(f-1)/2=1 n+2p-f+1=26+2*(1/2)-2+1=26  ==>28
layer2_pool=pooling(layer1_conv,[1,2,2,1],[1,2,2,1],'SAME')

node_nums=layer2_pool.get_shape().as_list()
weight_3=init_weight([node_nums[1]*node_nums[2]*node_nums[3],10])
bias_3=init_bias([10])
layer2_pool=tf.reshape(layer2_pool,[-1,node_nums[1]*node_nums[2]*node_nums[3]])
y=tf.nn.softmax(tf.matmul(layer2_pool,weight_3)+bias_3)

# 设置损失函数
cross_entropy=tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y),reduction_indices=[1]))
train_step=tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(cross_entropy)

correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
correct=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

init=tf.global_variables_initializer()
saver=tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    xs,ys=mnist.train.next_batch(BATCH_SIZE)
    print(xs.shape,ys.shape)
    for i in range(TRAINING_STEPS):
        _,correct_,cross_entropy_=sess.run([train_step,correct,cross_entropy],feed_dict={x:xs,y_:ys})
        if cross_entropy_ < 2.4:
            saver.save(sess,os.path.join(MODEL_PATH,MODEL_NAME))


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
(100, 784) (100, 10)


In [4]:
# L2正则化
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os

mnist=input_data.read_data_sets('MNIST_data/',one_hot=True)
BATCH_SIZE=100
INPUT=784
OUTPUT=10
NUM_CHANNELS=1
IMAGE=28
L2_REGULARIZER=0.01
LEARNING_RATE=0.001
TRAINING_STEPS=300
MODEL_PATH='CONV_Mnist'
MODEL_NAME='model.ckpt'
#初始化权重
weightList=[]
# 防止变量名重复
tf.reset_default_graph()
def init_weight(shape):
    weights=tf.Variable(tf.truncated_normal(shape,stddev=0.1))    
    weightList.append(tf.nn.l2_loss(weights))
    return weights

#初始化偏执
def init_bias(shape):
    bias=tf.Variable(tf.constant(0.1,shape=shape))
    return bias

#卷积层封装
def con2d(x,weights,strides,padding_type):
    return tf.nn.conv2d(x,weights,strides=strides,padding=padding_type)

#池化层
def pooling(x,kernel,strides,padding_type):
    return tf.nn.max_pool(x,ksize=kernel,strides=strides,padding=padding_type)

x=tf.placeholder(tf.float32,shape=[None,INPUT],name='x')
y_=tf.placeholder(tf.float32,shape=[None,OUTPUT],name='y')
x_=tf.reshape(x,[-1,IMAGE,IMAGE,NUM_CHANNELS])

#卷积 # 第一层p=(f-1)/2=2 n+2p-f+1=28+2*2-5+1=28
weights_1=init_weight([5,5,1,32])
bias=init_bias([32])
layer1_conv=tf.nn.relu(con2d(x_,weights_1,[1,1,1,1],'SAME')+bias)

#池化 # 第二层p=(f-1)/2=1 n+2p-f+1=26+2*(1/2)-2+1=26  ==>28
layer2_pool=pooling(layer1_conv,[1,2,2,1],[1,2,2,1],'SAME')

# 全连接 fcn
weight_3=init_weight([140*140*32,10])
bias_3=init_bias([10])
layer2_pool=tf.reshape(layer2_pool,[-1,140*140*32])
y=tf.nn.softmax(tf.matmul(layer2_pool,weight_3)+bias_3)

# 设置损失函数
cross_entropy=tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y),reduction_indices=[1]))

loss=tf.add_n(weightList)+cross_entropy
train_step=tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(loss)

correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
correct=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

init=tf.global_variables_initializer()
saver=tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    xs,ys=mnist.train.next_batch(BATCH_SIZE)
   
    for i in range(TRAINING_STEPS):
        _,correct_,cross_entropy_=sess.run([train_step,correct,cross_entropy],feed_dict={x:xs,y_:ys})        
        if i == 200:
            saver.save(sess,os.path.join(MODEL_PATH,MODEL_NAME))
            break



Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [11]:
#读取模型
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os

mnist=input_data.read_data_sets('MNIST_data/',one_hot=True)
BATCH_SIZE=100
INPUT=784
OUTPUT=10
NUM_CHANNELS=1
IMAGE=28
L2_REGULARIZER=0.01
LEARNING_RATE=0.001
TRAINING_STEPS=300
MODEL_PATH='CONV_Mnist'
MODEL_NAME='model.ckpt'
#初始化权重
weightList=[]
tf.reset_default_graph()
def init_weight(shape):
    weights=tf.Variable(tf.truncated_normal(shape,stddev=0.1))    
    weightList.append(tf.nn.l2_loss(weights))
    return weights

#初始化偏执
def init_bias(shape):
    bias=tf.Variable(tf.constant(0.1,shape=shape))
    return bias

#卷积层封装
def con2d(x,weights,strides,padding_type):
    return tf.nn.conv2d(x,weights,strides=strides,padding=padding_type)

#池化层
def pooling(x,kernel,strides,padding_type):
    return tf.nn.max_pool(x,ksize=kernel,strides=strides,padding=padding_type)

x=tf.placeholder(tf.float32,shape=[None,INPUT],name='x')
y_=tf.placeholder(tf.float32,shape=[None,OUTPUT],name='y')
x_=tf.reshape(x,[-1,IMAGE,IMAGE,NUM_CHANNELS])

#卷积 # 第一层p=(f-1)/2=2 n+2p-f+1=28+2*2-5+1=28
weights_1=init_weight([5,5,1,32])
bias=init_bias([32])
layer1_conv=tf.nn.relu(con2d(x_,weights_1,[1,1,1,1],'SAME')+bias)

#池化 # 第二层p=(f-1)/2=1 n+2p-f+1=26+2*(1/2)-2+1=26  ==>28
layer2_pool=pooling(layer1_conv,[1,2,2,1],[1,2,2,1],'SAME')

# 全连接 fcn
weight_3=init_weight([140*140*32,10])
bias_3=init_bias([10])
layer2_pool=tf.reshape(layer2_pool,[-1,140*140*32])
y=tf.nn.softmax(tf.matmul(layer2_pool,weight_3)+bias_3)

# 设置损失函数
cross_entropy=tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y),reduction_indices=[1]))

loss=tf.add_n(weightList)+cross_entropy
train_step=tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(loss)

correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
correct=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

saver=tf.train.Saver()
with tf.Session() as sess:
    xs,ys=mnist.train.next_batch(BATCH_SIZE)
    saver.restore(sess,'CONV_Mnist/model.ckpt')
    correct_=sess.run(correct,feed_dict={x:xs,y_:ys})
    print(correct_)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Restoring parameters from CONV_Mnist/model.ckpt
0.11


In [15]:
# 指数衰减
# 保存模型
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os

mnist=input_data.read_data_sets('MNIST_data/',one_hot=True)
BATCH_SIZE=100
INPUT=784
OUTPUT=10
NUM_CHANNELS=1
IMAGE=28

LEARNING_RATE=0.001
LEARNING_BASE=0.01
LEARN_DECAY=0.96

TRAINING_STEPS=300
MODEL_PATH='CONV_Mnist'
MODEL_NAME='model.ckpt'
#初始化权重
def init_weight(shape):
    weights=tf.Variable(tf.truncated_normal(shape,stddev=0.1))
    return weights

#初始化偏执
def init_bias(shape):
    bias=tf.Variable(tf.constant(0.1,shape=shape))
    return bias

#卷积层封装
def con2d(x,weights,strides,padding_type):
    return tf.nn.conv2d(x,weights,strides=strides,padding=padding_type)

#池化层
def pooling(x,kernel,strides,padding_type):
    return tf.nn.max_pool(x,ksize=kernel,strides=strides,padding=padding_type)

x=tf.placeholder(tf.float32,shape=[None,INPUT],name='x')
y_=tf.placeholder(tf.float32,shape=[None,OUTPUT],name='y')
x_=tf.reshape(x,[-1,IMAGE,IMAGE,NUM_CHANNELS])

#卷积 # 第一层p=(f-1)/2=2 n+2p-f+1=28+2*2-5+1=28
weights_1=init_weight([5,5,1,32])
bias=init_bias([32])
layer1_conv=tf.nn.relu(con2d(x_,weights_1,[1,1,1,1],'SAME')+bias)

#池化 # 第二层p=(f-1)/2=1 n+2p-f+1=26+2*(1/2)-2+1=26  ==>28
layer2_pool=pooling(layer1_conv,[1,2,2,1],[1,2,2,1],'SAME')


weight_3=init_weight([140*140*32,10])
bias_3=init_bias([10])
layer2_pool=tf.reshape(layer2_pool,[-1,140*140*32])
y=tf.nn.softmax(tf.matmul(layer2_pool,weight_3)+bias_3)

global_step=tf.Variable(0,trainable=False)
#设置学习率
learning_rate=tf.train.exponential_decay(LEARNING_BASE,global_step,mnist.train.num_examples/BATCH_SIZE,LEARN_DECAY)
# 设置损失函数
cross_entropy=tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y),reduction_indices=[1]))
train_step=tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy,global_step=global_step)

correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
correct=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

init=tf.global_variables_initializer()
saver=tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    xs,ys=mnist.train.next_batch(BATCH_SIZE)
    print(xs.shape,ys.shape)
    for i in range(TRAINING_STEPS):
        _,correct_,cross_entropy_=sess.run([train_step,correct,cross_entropy],feed_dict={x:xs,y_:ys})
    print(correct_)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
(100, 784) (100, 10)
0.12


In [13]:
# 滑动均值
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os

mnist=input_data.read_data_sets('MNIST_data/',one_hot=True)
BATCH_SIZE=100
INPUT=784
OUTPUT=10
NUM_CHANNELS=1
IMAGE=28
LEARNING_RATE=0.001
TRAINING_STEPS=300
MODEL_PATH='CONV_Mnist'
MODEL_NAME='model.ckpt'

MOVING_DACAY=0.99
#初始化权重
def init_weight(shape):
    weights=tf.Variable(tf.truncated_normal(shape,stddev=0.1))
    return weights

#初始化偏执
def init_bias(shape):
    bias=tf.Variable(tf.constant(0.1,shape=shape))
    return bias

#卷积层封装
def con2d(x,weights,strides,padding_type):
    return tf.nn.conv2d(x,weights,strides=strides,padding=padding_type)

#池化层
def pooling(x,kernel,strides,padding_type):
    return tf.nn.max_pool(x,ksize=kernel,strides=strides,padding=padding_type)

x=tf.placeholder(tf.float32,shape=[None,INPUT],name='x')
y_=tf.placeholder(tf.float32,shape=[None,OUTPUT],name='y')
x_=tf.reshape(x,[-1,IMAGE,IMAGE,NUM_CHANNELS])

#卷积 # 第一层p=(f-1)/2=2 n+2p-f+1=28+2*2-5+1=28
weights_1=init_weight([5,5,1,32])
bias=init_bias([32])
layer1_conv=tf.nn.relu(con2d(x_,weights_1,[1,1,1,1],'SAME')+bias)

#池化 #28
layer2_pool=pooling(layer1_conv,[1,2,2,1],[1,2,2,1],'SAME')


weight_3=init_weight([140*140*32,10])
bias_3=init_bias([10])
layer2_pool=tf.reshape(layer2_pool,[-1,140*140*32])
y=tf.nn.softmax(tf.matmul(layer2_pool,weight_3)+bias_3)

# 设置损失函数
gloal_step=tf.Variable(0,trainable=False)
ema=tf.train.ExponentialMovingAverage(MOVING_DACAY,gloal_step)
ema_op=ema.apply(tf.trainable_variables())

cross_entropy=tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y),reduction_indices=[1]))
train_step=tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(cross_entropy,global_step=gloal_step)

correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
correct=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.control_dependencies([ema_op,train_step]):
    train_op=tf.no_op(name='train')

init=tf.global_variables_initializer()
saver=tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    xs,ys=mnist.train.next_batch(BATCH_SIZE)
    print(xs.shape,ys.shape)
    for i in range(TRAINING_STEPS):
        _,correct_,cross_entropy_=sess.run([train_op,correct,cross_entropy],feed_dict={x:xs,y_:ys})
    print(correct_)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
(100, 784) (100, 10)
0.14


In [18]:
#使用dropout防止过拟合
mnist=input_data.read_data_sets('MNIST_data/',one_hot=True)
BATCH_SIZE=100
INPUT=784
OUTPUT=10
NUM_CHANNELS=1
IMAGE=28
LEARNING_RATE=0.001
TRAINING_STEPS=300

#初始化权重
def init_weight(shape):
    weights=tf.Variable(tf.truncated_normal(shape,stddev=0.1))
    return weights

#初始化偏执
def init_bias(shape):
    bias=tf.Variable(tf.constant(0.1,shape=shape))
    return bias

#卷积层封装
def con2d(x,weights,strides,padding_type):
    return tf.nn.conv2d(x,weights,strides=strides,padding=padding_type)

#池化层
def pooling(x,kernel,strides,padding_type):
    return tf.nn.max_pool(x,ksize=kernel,strides=strides,padding=padding_type)

x=tf.placeholder(tf.float32,shape=[None,INPUT],name='x')
y_=tf.placeholder(tf.float32,shape=[None,OUTPUT],name='y')
x_=tf.reshape(x,[-1,IMAGE,IMAGE,NUM_CHANNELS])

#卷积 # 第一层p=(f-1)/2=2 n+2p-f+1=28+2*2-5+1=28
weights_1=init_weight([5,5,1,32])
bias=init_bias([32])
layer1_conv=tf.nn.relu(con2d(x_,weights_1,[1,1,1,1],'SAME')+bias)

#池化 # 第二层p=(f-1)/2=1 n+2p-f+1=26+2*(1/2)-2+1=26  ==>28
layer2_pool=pooling(layer1_conv,[1,2,2,1],[1,2,2,1],'SAME')

# 再加一层卷积
weights_4=init_weight([3,3,1,64])
bias=init_bias([64])
layer4_conv=tf.nn.relu(con2d(x_,weights_4,[1,1,1,1],'SAME')+bias)
tf.nn.dropout(layer4_conv,0.8)
#全连接

node_nums=layer4_conv.get_shape().as_list()
weight_3=init_weight([node_nums[1]*node_nums[2]*node_nums[3],10])
bias_3=init_bias([10])
layer2_pool=tf.reshape(layer4_conv,[-1,node_nums[1]*node_nums[2]*node_nums[3]])
y=tf.nn.softmax(tf.matmul(layer2_pool,weight_3)+bias_3)

# 设置损失函数
cross_entropy=tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y),reduction_indices=[1]))
train_step=tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(cross_entropy)

correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
correct=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    xs,ys=mnist.train.next_batch(BATCH_SIZE)
    print(xs.shape,ys.shape)
    for i in range(TRAINING_STEPS):
        _,correct_,cross_entropy_=sess.run([train_step,correct,cross_entropy],feed_dict={x:xs,y_:ys})
    print(correct_,'-----------',cross_entropy_)
        
            

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
(100, 784) (100, 10)
0.69 ----------- 1.07192
